In [210]:
'''Felix Andersson, Janine de Vries, DV2626'''

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report, accuracy_score, mean_squared_error
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
import ast


In [211]:
# Steg 1: Ladda dataset
training_data = pd.read_csv("training_knn.csv")
exercise_data = pd.read_csv('gym.csv', delimiter=';')
important_data = exercise_data[['Exercise Name', 'Main_muscle', 'Mechanics', 'Target_Muscles', 'Synergist_Muscles', 'Stabilizer_Muscles', 'Antagonist_Muscles', 'Dynamic_Stabilizer_Muscles', 'Difficulty (1-5)', 'Secondary Muscles']].copy()
encode_columns = ['Main_muscle', 'Mechanics','Target_Muscles', 'Synergist_Muscles', 'Stabilizer_Muscles', 'Antagonist_Muscles', 'Dynamic_Stabilizer_Muscles', 'Secondary Muscles']
ID_columns = ["ID_Main_muscle", "ID_Mechanics", "ID_Target_Muscles", "ID_Synergist_Muscles", "ID_Stabilizer_Muscles", "ID_Antagonist_Muscles", "ID_Dynamic_Stabilizer_Muscles", "ID_Secondary Muscles"]

important_data.head()

,Exercise Name,Main_muscle,Mechanics,Target_Muscles,Synergist_Muscles,Stabilizer_Muscles,Antagonist_Muscles,Dynamic_Stabilizer_Muscles,Difficulty (1-5),Secondary Muscles
0,Front Raise,Shoulder,Isolated,"Anterior Deltoid,","Pectoralis Major, Clavicular, Deltoid, Lateral...","Trapezius, Upper, Levator Scapulae, Wrist Exte...",NaN,NaN,2,"Anterior Deltoid, Serratus Anterior"
1,Military Press,Shoulder,Compound,"Anterior Deltoid,","Pectoralis Major, Clavicular, Triceps Brachii,...","Trapezius, Upper, Levator Scapulae,",NaN,"Triceps, Long Head, Biceps Brachii, Short Head,",4,"Lateral Deltoid, Triceps Brachii"
2,Military Press: Seated,Shoulder,Compound,"Anterior Deltoid,","Deltoid, Lateral, Supraspinatus, Triceps Brach...","Trapezius, Upper, Levator Scapulae,",NaN,"Triceps, Long Head,",3,"Lateral Deltoid, Triceps Brachii"
3,Front Raise,Shoulder,Isolated,"Anterior Deltoid,","Pectoralis Major, Clavicular, Deltoid, Lateral...","Trapezius, Upper, Levator Scapulae, Wrist Exte...",NaN,NaN,2,"Anterior Deltoid, Serratus Anterior"
4,Front Raise: Alternating,Shoulder,Isolated,"Anterior Deltoid,","Pectoralis Major, Clavicular, Deltoid, Lateral...","Trapezius, Upper, Levator Scapulae, Wrist Exte...",NaN,NaN,2,"Anterior Deltoid, Serratus Anterior"


In [212]:

for column in encode_columns:
    important_data.loc[:, column] = important_data[column].fillna("")

    important_data.loc[:, column] = important_data[column].apply(lambda x: x.split(", ") if x else [])

    mlb = MultiLabelBinarizer()
    binary_matrix = mlb.fit_transform(important_data[column])

    important_data.loc[:, f"ID_{column}"] = binary_matrix.tolist()

    important_data = important_data.drop(columns=[column])

important_data.to_csv('gym_ID.csv', index=False)
important_data.head()

,Exercise Name,Difficulty (1-5),ID_Main_muscle,ID_Mechanics,ID_Target_Muscles,ID_Synergist_Muscles,ID_Stabilizer_Muscles,ID_Antagonist_Muscles,ID_Dynamic_Stabilizer_Muscles,ID_Secondary Muscles
0,Front Raise,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[0, 1]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Military Press,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[1, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,Military Press: Seated,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[1, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,Front Raise,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[0, 1]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Front Raise: Alternating,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[0, 1]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [213]:

for col in ID_columns:
    training_data[col] = training_data[col].apply(ast.literal_eval).apply(np.array)

flattened_features = []
for col in ID_columns:
    arrays = np.stack(training_data[col].values)  
    array_df = pd.DataFrame(arrays, columns=[f"{col}_{i}" for i in range(arrays.shape[1])])
    flattened_features.append(array_df)

data_flattened = pd.concat([training_data.drop(ID_columns, axis=1)] + flattened_features, axis=1)

X = data_flattened.drop(["Exercise Name", "Difficulty (1-5)", "score"], axis=1)
y = data_flattened["score"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_scaled, y)

data_test = pd.read_csv("gym_ID.csv")

for col in ID_columns:
    data_test[col] = data_test[col].apply(ast.literal_eval).apply(np.array)

flattened_features_test = []
for col in ID_columns:
    arrays_test = np.stack(data_test[col].values)
    array_df_test = pd.DataFrame(arrays_test, columns=[f"{col}_{i}" for i in range(arrays_test.shape[1])])
    flattened_features_test.append(array_df_test)

data_test_flattened = pd.concat([data_test.drop(ID_columns, axis=1)] + flattened_features_test, axis=1)

X_test = data_test_flattened.drop(["Exercise Name", "Difficulty (1-5)"], axis=1)
X_test_scaled = scaler.transform(X_test)

y_pred_test = knn.predict(X_test_scaled)

def adjust_score(predicted_scores, user_fitness_level, exercise_difficulty, max_difference=4):
    adjusted_scores = []
    for pred, difficulty in zip(predicted_scores, exercise_difficulty):
        factor = 1 - (abs(user_fitness_level - difficulty) / max_difference)**2
        factor = max(0.5,factor) 
        adjusted_scores.append(pred * factor) 
    return adjusted_scores

user_fitness_level = 5

exercise_difficulty = data_test_flattened["Difficulty (1-5)"]
adjusted_scores_test = adjust_score(y_pred_test, user_fitness_level, exercise_difficulty)

data_test_flattened["Predicted Score"] = adjusted_scores_test

sorted_results = data_test_flattened.sort_values(by="Predicted Score", ascending=False)

print("Recommended exercise: ", sorted_results["Exercise Name"].iloc[0])



Recommended exercise:  Deadlift
